# 0.Overview
In previous [my notebook](http://https://www.kaggle.com/astrung/recbole-lstm-for-recomendation), we tried to use GRU/LSTM model for testing effect of sequential model for recommendation with only iteration.
In this notebook, i showed how we can enhance sequential model with item features 

Due to memory limit and faster testing purpose, we will just use data in 2020.

If you want to use with all of interactions in all time, i have created a new atomic dataset here for you: 

* only interations data: https://www.kaggle.com/astrung/hm-atomic-interation
* iterations + item features data: https://www.kaggle.com/astrung/hm-atomic-interation-with-item-feature 

We also have other limit: we only train model and predict with users who buy more than 40 items and items which is bought by more than 40 people.

We will follow below steps for creating model:

1. In order to use Recbole, we create atomic file from interaction data and item data
2. Because we only use Recbole model for predicting with users who buy more than 40 items, other users will need to fill by default recomendation items. We create most viewed items in last month as defautl recomendation
3. We create dataset and train model in recbole.
4. We create prediction result by trained model
5. We combine recomendation result from most viewed items in last month and Recbole predicted model.

I will explain more detail in following cells.



# 1. Create atomic file

### 1.A create atomic of item features
we will create item features for feeding with iteration features into GRU4REC model 

In [26]:
import pandas as pd
df = pd.read_csv(r"articles.csv", dtype={'article_id': 'str'})
df.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [27]:
for col in df.columns:
    print(col)
    print(len(pd.unique(df[col])))

article_id
105542
product_code
47224
prod_name
45875
product_type_no
132
product_type_name
131
product_group_name
19
graphical_appearance_no
30
graphical_appearance_name
30
colour_group_code
50
colour_group_name
50
perceived_colour_value_id
8
perceived_colour_value_name
8
perceived_colour_master_id
20
perceived_colour_master_name
20
department_no
299
department_name
250
index_code
10
index_name
10
index_group_no
5
index_group_name
5
section_no
57
section_name
56
garment_group_no
21
garment_group_name
21
detail_desc
43405


#### we see so many couple of columns are [category_text, encoded_value]. So in order to avoid [Multicollinearity](https://link.springer.com/chapter/10.1007/978-0-585-25657-3_37), we will keep only one columns in each couple
We can see below couple of columns in item features, and we will keep one of them:

* use product_type_no - skip product_type_name
* use graphical_appearance_no - skip graphical_appearance_name
* use colour_group_code - skip colour_group_name
* use perceived_colour_value_id - skip perceived_colour_value_name
* use perceived_colour_master_id - skip perceived_colour_master_name
* use index_code - skip index_name
* use index_group_no - skip index_group_name
* use section_no - skip section_name
* use garment_group_no - skip garment_group_name
* use product_code, skip product_name
* use department_no, skip department_name

In [28]:
df = df.drop(columns = ['product_type_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name',
                        'perceived_colour_master_name', 'index_name', 'index_group_name', 'section_name', 
                        'garment_group_name', 'prod_name', 'department_name', 'detail_desc'])
df.head()

,article_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_code,index_group_no,section_no,garment_group_no
0,0108775015,108775,253,Garment Upper body,1010016,9,4,5,1676,A,1,16,1002
1,0108775044,108775,253,Garment Upper body,1010016,10,3,9,1676,A,1,16,1002
2,0108775051,108775,253,Garment Upper body,1010017,11,1,9,1676,A,1,16,1002
3,0110065001,110065,306,Underwear,1010016,9,4,5,1339,B,1,61,1017
4,0110065002,110065,306,Underwear,1010016,10,3,9,1339,B,1,61,1017


In [29]:
temp = df.rename(
    columns={'article_id': 'item_id:token', 'product_code': 'product_code:token', 'product_type_no': 'product_type_no:float',
             'product_group_name': 'product_group_name:token_seq', 'graphical_appearance_no': 'graphical_appearance_no:token', 
             'colour_group_code': 'colour_group_code:token', 'perceived_colour_value_id': 'perceived_colour_value_id:token', 
             'perceived_colour_master_id': 'perceived_colour_master_id:token', 'department_no': 'department_no:token', 
             'index_code': 'index_code:token', 'index_group_no': 'index_group_no:token', 'section_no': 'section_no:token', 
             'garment_group_no': 'garment_group_no:token'})
temp.head()

,item_id:token,product_code:token,product_type_no:float,product_group_name:token_seq,graphical_appearance_no:token,colour_group_code:token,perceived_colour_value_id:token,perceived_colour_master_id:token,department_no:token,index_code:token,index_group_no:token,section_no:token,garment_group_no:token
0,0108775015,108775,253,Garment Upper body,1010016,9,4,5,1676,A,1,16,1002
1,0108775044,108775,253,Garment Upper body,1010016,10,3,9,1676,A,1,16,1002
2,0108775051,108775,253,Garment Upper body,1010017,11,1,9,1676,A,1,16,1002
3,0110065001,110065,306,Underwear,1010016,9,4,5,1339,B,1,61,1017
4,0110065002,110065,306,Underwear,1010016,10,3,9,1339,B,1,61,1017


In [30]:
#!mkdir /kaggle/working/recbox_data
temp.to_csv(r'./recbox_data/recbox_data.item', index=False, sep='\t')

### 1.B create atomic of iteration features
we will create iteration features for GRU4REC model 

In [31]:
df = pd.read_csv(r"transactions_train.csv", dtype={'article_id': 'str'})
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [32]:
df['t_dat'] = pd.to_datetime(df['t_dat'], format="%Y-%m-%d")
df

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,0918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,0833459002,0.006763,1


In [33]:
import numpy as np
df['timestamp'] = df.t_dat.values.astype(np.int64) // 10 ** 9
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,timestamp
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2,1537401600
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2,1537401600
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2,1537401600
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2,1537401600
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2,1537401600


**We fill with data in only 2020(timestamp > > 1585620000) and create inter file**
For anyone need instruction about inter file, please check below links:
* https://recbole.io/docs/user_guide/data_intro.html
* https://recbole.io/docs/user_guide/data/atomic_files.html

if you want a full of iterations without limiting timestamp, please check here:
* 

In [34]:
temp = df[df['timestamp'] > 1580504400][['customer_id', 'article_id', 'timestamp']].rename(
    columns={'customer_id': 'user_id:token', 'article_id': 'item_id:token', 'timestamp': 'timestamp:float'})
temp

,user_id:token,item_id:token,timestamp:float
21884546,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,0843200001,1580515200
21884547,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,0830487001,1580515200
21884548,0006d3ff0caf0cb4d4e0615ee5cb7d268622364d483335...,0811777002,1580515200
21884549,0006d3ff0caf0cb4d4e0615ee5cb7d268622364d483335...,0689047004,1580515200
21884550,0006d3ff0caf0cb4d4e0615ee5cb7d268622364d483335...,0636505001,1580515200
...,...,...,...
31788319,fff2282977442e327b45d8c89afde25617d00124d0f999...,0929511001,1600732800
31788320,fff2282977442e327b45d8c89afde25617d00124d0f999...,0891322004,1600732800
31788321,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,0918325001,1600732800
31788322,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,0833459002,1600732800


We save atomic file in dataset format for using with recbole

In [35]:
temp.to_csv('./recbox_data/recbox_data.inter', index=False, sep='\t')

# 2. We create defautl recomendation for user who can not be predicted by sequential model.
I use this approach in notebook: https://www.kaggle.com/hervind/h-m-faster-trending-products-weekly You can check it for more detail information. I will juse copy only code here

In [36]:
import numpy as np
import pandas as pd 
from datetime import datetime, timedelta
import gc

In [37]:
N = 12
df  = pd.read_csv('transactions_train.csv',
                            usecols= ['t_dat', 'customer_id', 'article_id'], 
                            dtype={'article_id': 'int32', 't_dat': 'string', 'customer_id': 'string'})

In [38]:
df['customer_id'].head()

0    000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
1    000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
2    00007d2de826758b65a93dd24ce629ed66842531df6699...
3    00007d2de826758b65a93dd24ce629ed66842531df6699...
4    00007d2de826758b65a93dd24ce629ed66842531df6699...
Name: customer_id, dtype: string

In [39]:
df ['customer_id'] = df['customer_id'].str[-16:].apply(int, base=16)#.str[-16:].str.hex_to_int().astype('int64')

df['t_dat'] = pd.to_datetime(df['t_dat'])
last_ts = df['t_dat'].max()

In [40]:
%%time
tmp = df[['t_dat']].copy()
tmp['dow'] = tmp['t_dat'].dt.dayofweek
tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')

df['ldbw'] = tmp['ldbw'].values

CPU times: user 6.94 s, sys: 484 ms, total: 7.42 s
Wall time: 7.14 s


In [41]:
weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count().reset_index()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
df = df.merge(weekly_sales, on=['ldbw', 'article_id'], how = 'left')

In [42]:
weekly_sales = weekly_sales.reset_index().set_index('article_id')

df = df.merge(
    weekly_sales.loc[weekly_sales['ldbw']==last_ts, ['count']],
    on='article_id', suffixes=("", "_targ"))

df['count_targ'].fillna(0, inplace=True)
del weekly_sales

In [43]:
df['quotient'] = df['count_targ'] / df['count']
target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
general_pred = target_sales.nlargest(N).index.tolist()
general_pred = ['0' + str(article_id) for article_id in general_pred]
general_pred_str =  ' '.join(general_pred)
del target_sales

In [44]:
%%time
purchase_dict = {}

tmp = df.copy()#.to_pandas()
tmp['x'] = ((last_ts - tmp['t_dat']) / np.timedelta64(1, 'D')).astype(int)
tmp['dummy_1'] = 1 
tmp['x'] = tmp[["x", "dummy_1"]].max(axis=1)

a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
tmp['y'] = a / np.sqrt(tmp['x']) + b * np.exp(-c*tmp['x']) - d

tmp['dummy_0'] = 0 
tmp['y'] = tmp[["y", "dummy_0"]].max(axis=1)
tmp['value'] = tmp['quotient'] * tmp['y'] 

tmp = tmp.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
tmp = tmp.reset_index()

tmp = tmp.loc[tmp['value'] > 0]
tmp['rank'] = tmp.groupby("customer_id")["value"].rank("dense", ascending=False)
tmp = tmp.loc[tmp['rank'] <= 12]

# for customer_id in tmp['customer_id'].unique():
#     purchase_dict[customer_id] = {} 

# for customer_id, article_id, value in zip(tmp['customer_id'], tmp['article_id'], tmp['value']):
#     purchase_dict[customer_id][article_id] = value

purchase_df = tmp.sort_values(['customer_id', 'value'], ascending = False).reset_index(drop = True)
purchase_df['prediction'] = '0' + purchase_df['article_id'].astype(str) + ' '
purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
purchase_df['prediction'] = purchase_df['prediction'].str.strip()
#purchase_df = cudf.DataFrame(purchase_df)

CPU times: user 17.8 s, sys: 1.86 s, total: 19.6 s
Wall time: 19.1 s


In [45]:
%%time
sub  = pd.read_csv('sample_submission.csv', usecols= ['customer_id'], dtype={'customer_id': 'string'})

sub['customer_id2'] = sub['customer_id'].str[-16:].apply(int, base=16)

sub = sub.merge(purchase_df, left_on = 'customer_id2', right_on = 'customer_id', how = 'left',
               suffixes = ('', '_ignored'))

sub['prediction'] = sub['prediction'].fillna(general_pred_str)
sub['prediction'] = sub['prediction'] + ' ' +  general_pred_str
sub['prediction'] = sub['prediction'].str.strip()
sub['prediction'] = sub['prediction'].str[:131]
sub = sub[['customer_id', 'prediction']]
sub.to_csv(f'submission.csv',index=False)

CPU times: user 12 s, sys: 772 ms, total: 12.7 s
Wall time: 12.9 s


In [46]:
del sub
del purchase_df
del tmp
del general_pred
del df
del temp

# 3. Create dataset and train model with Recbole

For anyone need instruction document, please check this link: https://recbole.io/docs/user_guide/usage/use_modules.html

In [47]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, SASRec, NARM
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

for limiting memory and time traning, we will filter for only using user who bought more than 40 items and item which is sold more than 40 times. If you want to train with more data, please change below config
* user_inter_num_interval
* item_inter_num_interval

In [63]:
!python tuning.py > tuning_sasrec.txt

In [70]:
parameter_dict = {
    'data_path': '.',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp'],
                 'item': ['item_id', 'product_code', 'product_type_no', 'product_group_name', 
                          'graphical_appearance_no',
                      'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                      'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']
             },
    'selected_features': ['product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no',
                          'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                          'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no'],
    'neg_sampling': None,
    'epochs': 300,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'},
    'n_layers': 2
}

In [68]:
parameter_dict = {
    'data_path': '.',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp'],
                 'item': ['item_id', 'product_code', 'product_type_no', 'product_group_name', 
                          'graphical_appearance_no',
                      'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                      'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']
             },
    'selected_features': ['product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no',
                          'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                          'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no'],
    'neg_sampling': None,
    'epochs': 100,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'},
    'n_layers': 2
}

for lr in [0.01,0.005,0.001,0.0005,0.0001]:
    for dropout in ['[0.25,0.5]','[0.2,0.2]','[0.1,0.2]']:
        parameter_dict['learning_rate'] = lr
        parameter_dict['dropout_probs'] = dropout
        config = Config(model='', dataset='recbox_data', config_dict=parameter_dict)

        # init random seed
        init_seed(config['seed'], config['reproducibility'])

        # logger initialization
        init_logger(config)
        logger = getLogger()
        # Create handlers
        c_handler = logging.StreamHandler()
        c_handler.setLevel(logging.INFO)
        logger.addHandler(c_handler)

        # write config info into log
        logger.info(config)
        
        dataset = create_dataset(config)
        logger.info(dataset)
        
        train_data, valid_data, test_data = data_preparation(config, dataset)
        
        # model loading and initialization
        model = NARM(config, train_data.dataset).to(config['device'])
        logger.info(model)

        # trainer loading and initialization
        trainer = Trainer(config, model)

        # model training
        best_valid_score, best_valid_result = trainer.fit(train_data)

ModuleNotFoundError: No module named 'lightgbm'

In [71]:
parameter_dict['learning_rate'] = 0.001
parameter_dict['n_layers'] = 2
parameter_dict['n_heads'] = 1
config = Config(model='SASRec', dataset='recbox_data', config_dict=parameter_dict)

dataset = create_dataset(config)
#logger.info(dataset)

In [72]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

In [73]:
# model loading and initialization
model = SASRec(config, train_data.dataset).to(config['device'])
#logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

# 4. Create recommendation result from trained model

I note document here for any one want to customize it: https://recbole.io/docs/user_guide/usage/case_study.html

In [74]:
from recbole.utils.case_study import full_sort_topk
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [75]:
import torch


checkpoint = torch.load('/project/saved/SASRec-Mar-23-2022_16-58-56.pth', map_location=torch.device('cuda'))

model.load_state_dict(checkpoint['state_dict'])
model.to('cuda')

SASRec(
  (item_embedding): Embedding(12134, 64, padding_idx=0)
  (position_embedding): Embedding(50, 64)
  (trm_encoder): TransformerEncoder(
    (layer): ModuleList(
      (0): TransformerLayer(
        (multi_head_attention): MultiHeadAttention(
          (query): Linear(in_features=64, out_features=64, bias=True)
          (key): Linear(in_features=64, out_features=64, bias=True)
          (value): Linear(in_features=64, out_features=64, bias=True)
          (softmax): Softmax(dim=-1)
          (attn_dropout): Dropout(p=0.5, inplace=False)
          (dense): Linear(in_features=64, out_features=64, bias=True)
          (LayerNorm): LayerNorm((64,), eps=1e-12, elementwise_affine=True)
          (out_dropout): Dropout(p=0.5, inplace=False)
        )
        (feed_forward): FeedForward(
          (dense_1): Linear(in_features=64, out_features=256, bias=True)
          (dense_2): Linear(in_features=256, out_features=64, bias=True)
          (LayerNorm): LayerNorm((64,), eps=1e-12, eleme

In [76]:
from tqdm import tqdm

topk_items = []
for internal_user_id in tqdm(list(range(dataset.user_num))[1:]):
    _, topk_iid_list = full_sort_topk([internal_user_id], model, test_data, k=12, device=config['device'])
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)
print(len(topk_items))

100%|██████████████████████████████████████| 31350/31350 [01:39<00:00, 315.92it/s]

31350


In [77]:
external_item_str = [' '.join(x) for x in topk_items]
result = pd.DataFrame(external_user_ids, columns=['customer_id'])
result['prediction'] = external_item_str
result.head()

,customer_id,prediction
0,0006d3ff0caf0cb4d4e0615ee5cb7d268622364d483335...,0915529001 0915529003 0879088001 0903004003 07...
1,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,0448509014 0714790020 0448509018 0919365008 08...
2,0040e2fc2d1e7931a38355aca56b2c62b87e65051b7287...,0799365028 0448509014 0806131011 0714790020 07...
3,00ae9aa00317595e62a0f782228d3a98e017d071d03049...,0808648001 0879763001 0808684002 0852458002 08...
4,00daef2733086cfb0176a97646cb510472ca608f74c475...,0572998001 0572998009 0448509035 0928088001 04...


# 5. Combine result from most bought items and GRU model

In [78]:
submit_df = pd.read_csv('submission.csv')
submit_df.shape

(1371980, 2)

In [79]:
submit_df.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0745232001 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0723529001 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...


In [80]:
submit_df = pd.merge(submit_df, result, on='customer_id', how='outer')
submit_df.head()

,customer_id,prediction_x,prediction_y
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0745232001 07...,NaN
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0723529001 08...,NaN
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...,NaN
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...,NaN
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...,NaN


In [81]:
submit_df = submit_df.fillna(-1)
submit_df['prediction'] = submit_df.apply(
    lambda x: x['prediction_y'] if x['prediction_y'] != -1 else x['prediction_x'], axis=1)
submit_df.head()

,customer_id,prediction_x,prediction_y,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0745232001 07...,-1,0568601043 0568601006 0656719005 0745232001 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0723529001 08...,-1,0826211002 0800436010 0739590027 0723529001 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...,-1,0794321007 0852643001 0852643003 0858883002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...,-1,0448509014 0573085028 0751471001 0706016001 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...,-1,0730683050 0791587015 0896152002 0818320001 09...


In [82]:

submit_df = submit_df.drop(columns=['prediction_y', 'prediction_x'])
submit_df.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0745232001 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0739590027 0723529001 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...


In [83]:
submit_df.to_csv('sasrec_001lr_2layers_1head.csv', index=False)